<!--[![Binder](https://mybinder.org/badge_logo.svg)](https://nbviewer.org/github/Sistemas-Multimedia/Sistemas-Multimedia.github.io/blob/master/contents/PNG/display_video.ipynb)-->

[![Binder](https://mybinder.org/badge_logo.svg)](https://mybinder.org/v2/gh/Sistemas-Multimedia/Sistemas-Multimedia.github.io/91ce476f6f6d656457fd707eeddbc96cf72864d8?urlpath=lab%2Ftree%2Fcontents%2FPNG%2Fdisplay_video.ipynb)

[![Colab](https://badgen.net/badge/Launch/on%20Google%20Colab/blue?icon=notebook)](https://colab.research.google.com/github/Sistemas-Multimedia/Sistemas-Multimedia.github.io/blob/master/contents/PNG/display_video.ipynb)

# Display a sequence of video

In [ ]:
%%bash
if [ -d "$HOME/repos" ]; then
    echo "\"$HOME/repos\" exists"
else
    mkdir ~/repos
    echo Created $HOME/repos
fi

In [ ]:
%%bash
if [ -d "$HOME/repos/MRVC" ]; then
    cd $HOME/repos/MRVC
    echo "$HOME/repos/MRVC ... "
    git pull 
else
    cd $HOME/repos
    git clone https://github.com/Sistemas-Multimedia/MRVC.git
fi

In [ ]:
%%bash
if [ -d "$HOME/repos/image_IO" ]; then
    cd $HOME/repos/image_IO
    echo "$HOME/repos/image_IO ... "
    git pull 
else
    cd $HOME/repos
    git clone https://github.com/vicente-gonzalez-ruiz/image_IO.git
fi

In [ ]:
# Please, note that here the packages that are used by the libraries
# must also be installed.

import os

try:
    import colored
except:
    !pip install colored

try:
    import numpy as np
except:
    !pip install numpy

try:
    import cv2
except:
    !pip install opencv-python
    !pip install opencv-python-headless # Binder compatibility

try:
    import matplotlib.pyplot as plt
    %matplotlib inline
except:
    !pip install matplotlib

!ln -sf ~/repos/image_IO/image_3.py .
!ln -sf ~/repos/image_IO/logging_config.py .

import image_3 as frame

In [ ]:
if 'google.colab' in str(get_ipython()):
    !apt-get -qq install -y bc ffmpeg
# Binder install the pakages indicated in "apt.txt".

In [ ]:
def show(img, title):
    img = frame.normalize(img)
    plt.figure(figsize=(10,10))
    plt.title(title, fontsize=20)
    plt.imshow(img)

def show_seq(prefix="/tmp/", first_frame = 0, N_frames=5):
    for i in range(first_frame, first_frame + N_frames):
        img = frame.read(prefix, i)
        show(img, f"{prefix}{i:03d}")

def show_motion(prefix="/tmp/", first_frame = 0, N_frames=5):
    plt.figure(figsize=(10,10))
    prev = frame.read(prefix, first_frame)
    plt.title(f"{prefix}{first_frame:03d}", fontsize=20)
    plt.imshow(frame.normalize(prev))
    for i in range(first_frame + 1, first_frame + N_frames, 1):
        img = frame.read(prefix, i)
        diff = img.astype(np.int16) - prev
        prev = img
        diff = frame.normalize(diff)
        plt.figure(figsize=(10,10))
        plt.title(f"{prefix}{i:03d}", fontsize=20)
        plt.imshow(diff)
        #plt.imshow(diff[0])

In [ ]:
home = os.environ["HOME"]

In [ ]:
!rm -f /tmp/*.png

In [ ]:
sequence = "stockholm_1280x768x50x420x578.avi"
URL = "http://www.hpca.ual.es/~vruiz/videos/"
N_frames = 5 # Min val 3
first_frame = 2

In [ ]:
prefix="/tmp/original_"
!$home/repos/MRVC/sequences/extract_frames.sh -u $URL -s $sequence -o $prefix -f $first_frame -n $N_frames

In [ ]:
#home = os.environ["HOME"]
#sequence_path = home + "/repos/MRVC/sequences/stockholm/"

In [ ]:
img = frame.read(prefix, 2)
show(img, "stockholm002")

In [ ]:
show_seq(prefix, first_frame = first_frame)

In [ ]:
show_motion(prefix, first_frame = first_frame)